In [1]:
import torch

from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    MBartTokenizer,
    MBartForConditionalGeneration,
    BertForTokenClassification,
    AutoModelForSequenceClassification,
)
from nltk import sent_tokenize

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
class_model = AutoModelForSequenceClassification.from_pretrained(
    "../models/tiny_rubert_fine_tune/tiny_rubert_finetuned_v1", num_labels=6
).to(device)

In [4]:
ext_sum_tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/rubert_ext_sum_gazeta")
sep_token = ext_sum_tokenizer.sep_token
sep_token_id = ext_sum_tokenizer.sep_token_id
ext_sum_model = BertForTokenClassification.from_pretrained("IlyaGusev/rubert_ext_sum_gazeta").to(device)

In [5]:
def get_sentances(text):
    paragraphs = []
    for paragraph in text.replace("\r\n", "\n").split("\n\n"):
        if len(paragraph.strip()) > 0:
            paragraphs.append(sent_tokenize(paragraph.strip()))
    
    return [sentance for paragraph in paragraphs for sentance in paragraph]

In [169]:
sents = get_sentances(text)

In [171]:
def get_cat(sentances, model=class_model, tokenizer=class_tokenizer, cats=6):
    weighted_sentances = [{} for i in range(cats)]
    final_sentances = []
    
    for sent in sentances:
        
        inputs = tokenizer(
        sent, padding=True, truncation=True, return_tensors="pt"
        ).to(device)
        logits = model(**inputs).logits
        
        if logits.max().item() < 5:
            continue
        
        pred_cat = logits.argmax().item()
        weighted_sentances[pred_cat][sent] = logits.max().item()
        
    for ind, weighted_sent in enumerate(weighted_sentances):
        if len(weighted_sent) > 5:
            truncated_sentances = [key for key, val in sorted(weighted_sentances[ind].items(), 
                                                            key=lambda x: x[1], reverse=True)][:5]
            
        else:
            truncated_sentances = [key for key, val in sorted(weighted_sentances[ind].items(), 
                                                            key=lambda x: x[1], reverse=True)]
        
        final_sentances.append(truncated_sentances)
    
    return final_sentances
    

In [172]:
sents_labeled = get_cat(sents)

In [173]:
sents_labeled[2]

['К недостаткам — низкую стабильность черного к отклонению взгляда от перпендикуляра к плоскости экрана, агрессивную неотключаемую динамическую подстройку яркости и перенасыщенные цвета.',
 'Вообще, такая неотключаемая коррекция яркости ничего кроме вреда не приносит, поскольку постоянная смена яркости экрана как минимум может вызывать некоторый дискомфорт, снижает различимость градаций в тенях в случае темных изображений и читаемость экрана на ярком свету, так как на не самых светлых в среднем изображениях яркость подсветки существенно занижается.',
 'Приведем фотографии, на которых на экраны смартфона и Nexus 7 выведены одинаковые изображения, при этом яркость экранов изначально установлена примерно на 200 кд/м², а цветовой баланс на фотоаппарате принудительно переключен на 6500 К.\nПерпендикулярно к экранам белое поле:',
 'При ручном управлении яркостью и при выводе белого поля во весь экран максимальное значение яркости составило 550 кд/м².',
 'Спектры компонент достаточно хорошо р

In [174]:
def ext_summarize(text, tokenizer=ext_sum_tokenizer, model=ext_sum_model):
    sentences = [s for s in sent_tokenize(text)]
    article_text = sep_token.join(sentences)

    inputs = tokenizer(
        [article_text],
        max_length=3000,
        padding=True,
        truncation=True,
        return_tensors="pt",
    ).to(device)
    sep_mask = inputs["input_ids"][0] == sep_token_id

    # Fix token_type_ids
    current_token_type_id = 0 
    for pos, input_id in enumerate(inputs["input_ids"][0]):
        inputs["token_type_ids"][0][pos] = current_token_type_id
        if input_id == sep_token_id:
            current_token_type_id = 1 - current_token_type_id

    # Infer model
    with torch.no_grad(): 
        outputs = model(**inputs) 
    logits = outputs.logits[0, :, 1]
    
    logits = logits[sep_mask]
    logits, indices = logits.sort(descending=True)
    logits, indices = logits.cpu().tolist(), indices.cpu().tolist()
    pairs = list(zip(logits, indices))
    pairs = pairs[:3]
    indices = list(sorted([idx for _, idx in pairs]))
    summary = " ".join([sentences[idx] for idx in indices])
    return summary

In [176]:
ext_summarize(" ".join(sents_labeled[0]))

'От комплектного сетевого зарядного устройства мощностью 45 Вт смартфон полностью заряжается за 1 час. Время автономной работы\nСмартфон Tecno Pova 5 получил батарею с увеличенным объемом 6000 мА·ч, однако в тестах продемонстрировал примерно такую же автономность, как многие смартфоны с батареей в 5000 мА·ч. Комплект включает зарядное устройство с максимальной выходной мощностью 45 Вт, соединительный кабель, защитную пленку на экран и гибкий прозрачный защитный чехол.'

In [177]:
sents_labeled[5]

['В этом смартфоне установлены стереодинамики, и они выдают максимально громкий и вполне достойный насыщенный яркий звук с широким диапазоном частот.',
 'На нижнем торце установлены динамик, микрофон, разъем USB Type-C и 3,5-миллиметровый аудиовыход.',
 'Также сохранен 3,5-миллиметровый аудиовыход для проводных наушников.',
 'Есть функция записи телефонных разговоров с линии.',
 'Все необходимые встроенные датчики, вплоть до гироскопа, на месте.']

In [162]:
texts = []
for i in range(6):
    texts.append(ext_summarize(" ".join(sents_labeled[i])))
    # texts.append(generate(mbart_tokenizer, mbart_model, " ".join(sents_labeled[i])))

In [163]:
texts

['Беспроводная зарядка не поддерживается. От комплектного сетевого зарядного устройства мощностью 45 Вт смартфон полностью заряжается за 1 час. Время автономной работы\nСмартфон Tecno Pova 5 получил батарею с увеличенным объемом 6000 мА·ч, однако в тестах продемонстрировал примерно такую же автономность, как многие смартфоны с батареей в 5000 мА·ч.',
 'Детализация не такая высокая, как хотелось бы: изображение рыхловатое, по качеству не сильно лучше, чем 1080р. И тестовая картинка (профиль Оригинальный цвет): Разница в деталях едва заметна, и в целом лучше смотрятся даже обработанные 12,5-мегапиксельные фото с искусственно увеличенной резкостью, в то время как полноразмерная картинка 50 Мп выглядит мыльной и полна мелких артефактов. По умолчанию снимки делаются в разрешении 12,5 Мп с функцией объединения пикселей (4 в 1).',
 'К недостаткам — низкую стабильность черного к отклонению взгляда от перпендикуляра к плоскости экрана, агрессивную неотключаемую динамическую подстройку яркости и

In [201]:
generate(mbart_tokenizer, mbart_model, " ".join(texts))

'Смартфон Tecno Pova 5 получил увеличенную батарею с увеличенным объемом 6000 мА·ч, однако в тестах продемонстрировал примерно такую же автономность, как многие смартфоны с батареей в 5000 мА·ч. Детализация не такая высокая, как хотелось бы, и в целом лучше смотрятся даже обработанные 12,5-мегапиксельные фото с искусственно увеличенной резкостью, в то время как полноразмерная картинка 50 Мп выглядит мыльной и полна мелких артефактов.'

In [185]:
texts

['Беспроводная зарядка не поддерживается. От комплектного сетевого зарядного устройства мощностью 45 Вт смартфон полностью заряжается за 1 час. Время автономной работы\nСмартфон Tecno Pova 5 получил батарею с увеличенным объемом 6000 мА·ч, однако в тестах продемонстрировал примерно такую же автономность, как многие смартфоны с батареей в 5000 мА·ч.',
 'Детализация не такая высокая, как хотелось бы: изображение рыхловатое, по качеству не сильно лучше, чем 1080р. И тестовая картинка (профиль Оригинальный цвет): Разница в деталях едва заметна, и в целом лучше смотрятся даже обработанные 12,5-мегапиксельные фото с искусственно увеличенной резкостью, в то время как полноразмерная картинка 50 Мп выглядит мыльной и полна мелких артефактов. По умолчанию снимки делаются в разрешении 12,5 Мп с функцией объединения пикселей (4 в 1).',
 'К недостаткам — низкую стабильность черного к отклонению взгляда от перпендикуляра к плоскости экрана, агрессивную неотключаемую динамическую подстройку яркости и

In [100]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
mbart_tokenizer = MBartTokenizer.from_pretrained(model_name)
mbart_model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

In [200]:
def generate(tokenizer, model, article_text):
    input_ids = tokenizer(
    [article_text],
    max_length=3000,
    add_special_tokens=True,
    padding=True,
    truncation=True,
    return_tensors="pt"
    )["input_ids"].to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=6,
        max_length=200
    
      
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

In [81]:

from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_sum_gazeta"
t5_tokenizer = AutoTokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

In [153]:
ext_summarize(summar)

'\n\nВ конечном итоге, у Honor получилась интересная модель для рынка, она стоит своих денег и предлагает отличное качество связи (об этом нынче принято забывать говорить). Интересная камера, дополнительные возможности, быстрая зарядка в комплекте – смартфон станет популярным на российском рынке (продажи в Китае начались ранее, и там смартфон крайне популярен).'

In [155]:
generate(mbart_tokenizer, mbart_model, texts[1])

В сети появилось видео, на котором показано, как выглядят обработанные 12,5-мегапиксельные фото с искусственно увеличенной резкостью, в то время как полноразмерная картинка 50 Мп выглядит мыльной и полна мелких артефактов.


In [134]:
generate(t5_tokenizer, t5_model, texts[0])

Смартфон Tecno Pova 5 продемонстрировал примерно такую же автономность, как многие другие смартфоны с батареей в 6000 мАбудьч. В тестах смартфоны продемонстрировали такую же автономность как многие смартфоны с батареей с увеличенным объемом 6000 мАбайсч.
